In [0]:
import requests
import json
from datetime import datetime, timedelta
from web3 import Web3
import pandas as pd

hoje = datetime.utcnow()
inicio = (hoje - timedelta(days=30)).strftime("%Y-%m-%d")
fim = hoje.strftime("%Y-%m-%d")

print(f"inicio: {inicio}") 
print(f"fim: {fim}")

usdc_checksum = Web3.to_checksum_address("0xA0b86991c6218b36c1d19d4a2e9Eb0cE3606eB48")

access_token = "ory_at_dVus-KX5ztTduh_cX-fbCrswhHCVjCyZ1wkMskhRL6c.70NmAvuT6kyJ0bWfrii3Vu2Bs4YP6X3_E9YOnq7oUnQ"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


networks = ["ethereum", "matic", "fantom", "avalanche", "klaytn"]

url = "https://graphql.bitquery.io"

dfs = []

for network in networks:
    print(f"🔍 Coletando dados para: {network}")

    query = f'''
    {{
    ethereum(network: {network}) {{
        dexTrades(
        date: {{after: "{inicio}", till: "{fim}"}},
        tradeAmountUsd: {{gt: 30000}},
        options: {{limit: 10000}}
        ) {{
        date {{
            date
        }}
        baseCurrency {{
            symbol
            name
            address
        }}
        quoteCurrency {{
            symbol
            name
            address
        }}
        buyCurrency {{
            symbol
        }}
        sellCurrency {{
            symbol
            name
            address
            tokenId
        }}
        exchange {{
            name
            fullName
        }}
        tradeAmount(in: USD)
        buyAmount
        sellAmount
        baseAmount
        quotePrice
        price
        trades: count
        protocol
        side
        gas
        gasPrice
        gasValue
        smartContract {{
            contractType
            currency {{
            name
            symbol
            }}
        }}
        }}
    }}
    }}
    '''

    try:
        response = requests.post(url, headers=headers, data=json.dumps({"query": query}))
        print("📝 Status:", response.status_code)
        print("📝 Resposta crua:", response.text[:300])
        json_response = response.json()

        trades = json_response.get("data", {}).get("ethereum", {}).get("dexTrades", [])
        if trades and isinstance(trades, list):
            df = pd.DataFrame(trades)
            df["network"] = network
            dfs.append(df)
        else:
            print(f"⚠️ Nenhum dado retornado para: {network}")
    except Exception as e:
        print(f"❌ Erro em {network}: {e}")

if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    df_spark = spark.createDataFrame(df_total)
    display(df_spark)
else:
    print("⚠️ Nenhum dado foi coletado de nenhuma rede.")


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace")
df_spark.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("workspace.ethereum.dex_trades_raw")